In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv("../data/HMIS_DATA_CORRECTED_17_21/mh_dist17_21_with_IDs_date_correction.csv")
data = data[(data['indicator_type'] == 'Total [(A+B) or (C+D)]')]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)

In [3]:
def prophet_forecast(
    series,
    district_name,
    forecast_periods=12,
    yearly_seasonality='auto',
    weekly_seasonality=False,
    daily_seasonality=False,
    changepoint_prior_scale=0.05,
    seasonality_prior_scale=10.0,
    holidays=None,
    plot_forecast=True
):
    """
    Prophet time series forecasting with error handling and RMSE calculation.
    Returns (metrics, forecast_df) or (None, None) on error.
    """
    try:
        # 0. Directory setup
        os.makedirs('Prophet_Forecasts', exist_ok=True)

        # 1. Prepare DataFrame for Prophet
        df = pd.DataFrame({'ds': series.index, 'y': series.values})
        df = df.dropna()
        if len(df) < forecast_periods + 12:
            raise ValueError(f"Series too short ({len(df)} points) for {forecast_periods}-step forecast.")

        # 2. Train/test split
        train = df.iloc[:-forecast_periods]
        test = df.iloc[-forecast_periods:]

        # 3. Model setup
        model = Prophet(
            yearly_seasonality=yearly_seasonality,
            weekly_seasonality=weekly_seasonality,
            daily_seasonality=daily_seasonality,
            changepoint_prior_scale=changepoint_prior_scale,
            seasonality_prior_scale=seasonality_prior_scale,
            holidays=holidays
        )
        model.fit(train)

        # 4. Forecast
        future = model.make_future_dataframe(periods=forecast_periods, freq='MS')
        forecast = model.predict(future)

        # 5. Extract forecasted values for test period
        forecast_test = forecast.set_index('ds').loc[test['ds']]
        y_pred = forecast_test['yhat'].values
        y_test = test['y'].values

        # 6. Metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        metrics = {
            'district': district_name,
            'rmse': rmse,
            'forecast_periods': forecast_periods
        }

        # 7. Save forecast DataFrame
        forecast_df = pd.DataFrame({
            'district': district_name,
            'date': test['ds'].values,
            'actual': y_test,
            'forecast': y_pred
        })
        forecast_df.to_csv(
            os.path.join('Prophet_Forecasts', f'{district_name}_forecast.csv'),
            index=False
        )

        # 8. Plotting
        if plot_forecast:
            plt.figure(figsize=(12, 6))
            plt.plot(df['ds'], df['y'], label='Original')
            plt.plot(forecast_df['date'], forecast_df['forecast'], label='Prophet Forecast', color='crimson')
            plt.scatter(forecast_df['date'], forecast_df['actual'], label='Actual (Test)', color='black')
            plt.title(f'{district_name} Prophet Forecast (RMSE: {rmse:.2f})')
            plt.legend()
            plt.savefig(os.path.join('Prophet_Forecasts', f'{district_name}_forecast_plot.png'))
            plt.close()

        return metrics, forecast_df

    except Exception as e:
        print(f"⚠️ Skipping {district_name}: {str(e)}")
        return None, None

In [4]:
districts = data['district'].unique()
rmse_values = []

# Usage Example


def run_for_each_district():
    results = {}
    
    for district in districts:
        district_data = data[data['district'] == district]
        ts = district_data["I8"].asfreq('MS')
        
        metrics, forecast = prophet_forecast(
            series=ts,
            district_name=district,
            forecast_periods=6,  # Forecast the last year
            yearly_seasonality='auto',
            weekly_seasonality=False,
            daily_seasonality=False
        )

        
    
        # Show results
        print("=== Metrics ===")
        if metrics:
            print(metrics)
        print("\n=== Forecast Data ===")
        print(forecast)
    
    return results
run_for_each_district()

20:54:35 - cmdstanpy - INFO - Chain [1] start processing
20:54:35 - cmdstanpy - INFO - Chain [1] done processing


=== Metrics ===
{'district': 'AHMEDNAGAR', 'rmse': 590.3203039501782, 'forecast_periods': 6}

=== Forecast Data ===
     district       date  actual     forecast
0  AHMEDNAGAR 2020-10-01    5719  6306.582435
1  AHMEDNAGAR 2020-11-01    5427  6344.190162
2  AHMEDNAGAR 2020-12-01    6256  5877.522820
3  AHMEDNAGAR 2021-01-01    6196  5732.465008
4  AHMEDNAGAR 2021-02-01    6458  5763.197658
5  AHMEDNAGAR 2021-03-01    5735  5482.973220


20:54:36 - cmdstanpy - INFO - Chain [1] start processing
20:54:36 - cmdstanpy - INFO - Chain [1] done processing
20:54:37 - cmdstanpy - INFO - Chain [1] start processing


=== Metrics ===
{'district': 'AKOLA', 'rmse': 292.66665154617465, 'forecast_periods': 6}

=== Forecast Data ===
  district       date  actual     forecast
0    AKOLA 2020-10-01    1395  1654.121687
1    AKOLA 2020-11-01    1204  1851.770157
2    AKOLA 2020-12-01    1630  1483.619319
3    AKOLA 2021-01-01    1745  1759.946936
4    AKOLA 2021-02-01    1618  1554.445837
5    AKOLA 2021-03-01    1782  1743.496204


20:54:37 - cmdstanpy - INFO - Chain [1] done processing
20:54:38 - cmdstanpy - INFO - Chain [1] start processing


...

20:55:11 - cmdstanpy - INFO - Chain [1] done processing
20:55:12 - cmdstanpy - INFO - Chain [1] start processing


=== Metrics ===
{'district': 'WASHIM', 'rmse': 233.2314068144828, 'forecast_periods': 6}

=== Forecast Data ===
  district       date  actual     forecast
0   WASHIM 2020-10-01    1304  1400.755373
1   WASHIM 2020-11-01    1237  1221.748478
2   WASHIM 2020-12-01    1499  1120.748384
3   WASHIM 2021-01-01    1392  1091.362239
4   WASHIM 2021-02-01    1584  1580.020715
5   WASHIM 2021-03-01    1701  1412.358457


20:55:12 - cmdstanpy - INFO - Chain [1] done processing


=== Metrics ===
{'district': 'YAVATMAL', 'rmse': 411.44643646778434, 'forecast_periods': 6}

=== Forecast Data ===
   district       date  actual     forecast
0  YAVATMAL 2020-10-01    2797  2696.429684
1  YAVATMAL 2020-11-01    2682  3426.640505
2  YAVATMAL 2020-12-01    3140  3077.276885
3  YAVATMAL 2021-01-01    3105  2632.429592
4  YAVATMAL 2021-02-01    3242  2939.704837
5  YAVATMAL 2021-03-01    3717  3353.014004
⚠️ Skipping MUMBAI SUBURBAN: Series too short (12 points) for 6-step forecast.
=== Metrics ===

=== Forecast Data ===
None


{}

In [5]:
districts

array(['AHMEDNAGAR', 'AKOLA', 'AMRAVATI', 'AURANGABAD', 'BEED',
       'BHANDARA', 'BULDHANA', 'CHANDRAPUR', 'DHULE', 'GADCHIROLI',
       'GONDIA', 'HINGOLI', 'JALGAON', 'JALNA', 'KOLHAPUR', 'LATUR',
       'MUMBAI', 'NAGPUR', 'NANDED', 'NANDURBAR', 'NASHIK', 'OSMANABAD',
       'PALGHAR', 'PARBHANI', 'PUNE', 'RAIGAD', 'RATNAGIRI', 'SANGLI',
       'SATARA', 'SINDHUDURG', 'SOLAPUR', 'THANE', 'WARDHA', 'WASHIM',
       'YAVATMAL', 'MUMBAI SUBURBAN'], dtype=object)